define data format

In [1]:
import csv
train_labels, train_images, test_labels, test_images = ([] for _ in range(4))
classes = 47
unit = [[0] * _ + [1] + [0] * (classes - 1 - _) for _ in range(classes)]
# print(*unit, sep='\n')

## reading
read data from csv file

In [2]:
with open('EMNIST_data/emnist-balanced-train.csv','r') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:
        train_labels.append(unit[int(row[0])])
        train_images.append(list(map(lambda x:int(x)/255.0, row[1:])))
with open('EMNIST_data/emnist-balanced-test.csv','r') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:
        test_labels.append(unit[int(row[0])])
        test_images.append(list(map(lambda x:int(x)/255.0, row[1:])))

## loading
load data into numpy

In [3]:
import numpy as np
train_labels, train_images = np.asarray(train_labels), np.asarray(train_images)
test_labels, test_images = np.asarray(test_labels), np.asarray(test_images)
print(len(train_labels))
print(train_labels[0].size)
print(train_images[0].size)

112800
47
784


## modeling
create a simple linear model classifier

In [4]:
import tensorflow as tf
x = tf.placeholder("float", [None, 784])
W = tf.Variable(tf.zeros([784,classes]))
b = tf.Variable(tf.zeros([classes]))
y = tf.nn.softmax(tf.matmul(x, W) + b)

In [5]:
y_ = tf.placeholder("float", [None,classes])
cross_entropy = -tf.reduce_sum(y_*tf.log(y))
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)

## training
using batch gradient decsent to train the model with 10000 iterations

In [6]:
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()
batch_size = 100
for _ in range(10000):
    if _ % 100 == 0:
        print(_,end=' ')
    d1 = (_ * batch_size) % 100000
    d2 = d1 + batch_size
    sess.run(train_step, feed_dict={
        x: train_images[d1 : d2], 
        y_: train_labels[d1 : d2]
    })


0 100 200 300 400 500 600 700 800 900 1000 1100 1200 1300 1400 1500 1600 1700 1800 1900 2000 2100 2200 2300 2400 2500 2600 2700 2800 2900 3000 3100 3200 3300 3400 3500 3600 3700 3800 3900 4000 4100 4200 4300 4400 4500 4600 4700 4800 4900 5000 5100 5200 5300 5400 5500 5600 5700 5800 5900 6000 6100 6200 6300 6400 6500 6600 6700 6800 6900 7000 7100 7200 7300 7400 7500 7600 7700 7800 7900 8000 8100 8200 8300 8400 8500 8600 8700 8800 8900 9000 9100 9200 9300 9400 9500 9600 9700 9800 9900 

## evaluating

In [7]:
print('weight:', b.eval())
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print('accuracy:', sess.run(accuracy, feed_dict={x: test_images, y_: test_labels}))

weight: [-1.5820951   5.13485     0.1543055  -1.8029169   1.0015433   1.6056659
 -2.9833453   0.98182875 -3.6977134  -1.5393919  -1.585996   -4.238481
  1.3872472  -1.0188286  -2.6760602   1.570452   -2.0040183  -0.5062683
  3.935151    5.5065475   0.03309623  7.1707253  -1.909832   -0.12458351
 -2.3510158  -0.1997416  -3.0293717  -3.14986     0.29784274  3.5317054
 -0.03364437  2.1130202  -2.542801   -1.5513492   3.8119006  -0.5839036
 -2.0647957  -0.4404019  -0.21600418 -2.735401    0.5803906  -1.9214307
  3.462625    0.6747457  -2.426396    1.9436027   4.018545  ]
accuracy: 0.6549468
